# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

## 1. 环境准备与DeepSeek API配置

In [2]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

## 2. 需求拆解与Agent任务规划

#### 用户痛点与核心需求：
*   **效率低下：** 人工创作周期长，难以满足高频发布需求。
*   **创意瓶颈：** 难以持续产出新颖、吸引人的爆款创意。
*   **趋势捕捉难：** 实时流行元素难以快速融入文案。
*   **平台特性把握：** 小红书特有风格（标题、正文、标签、表情）难以精准复制。

#### “爆款”文案的特征：
*   **强吸引力标题：** 制造好奇、痛点共鸣、利益点清晰。
*   **沉浸式正文：** 真实体验分享、细节描述、情感共鸣。
*   **精准且多样标签：** 热门话题、品牌词、产品词、垂直领域词。
*   **生动表情符号：** 增强表达力，提升活泼感。
*   **清晰的行动召唤 (CTA)。**

#### Agent 任务规划：核心工作流
1.  **用户指令接收：** 接收产品信息、主题、风格等。
2.  **信息收集 (Web Search/DB Query)：** 实时搜索行业趋势、热门话题、竞品分析、产品卖点。
3.  **内容构思与初稿生成 (LLM)：** 结合所有信息，撰写标题、正文、标签、表情符号。
4.  **风格与格式优化 (LLM)：** 根据小红书平台特点和指定风格，对文案进行润色和结构调整。
5.  **最终输出：** 呈现完整文案。

## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词)

System Prompt 是 Agent 的“大脑”和“行为准则”。它定义了 Agent 的角色、目标以及工作方式。我们将采用 `Thought-Action-Observation` (ReAct) 模式来引导 DeepSeek 的推理过程。

In [3]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [4]:
# 钢琴培训专用的系统提示词
PIANO_TRAINING_SYSTEM_PROMPT = """
你是一个专业的钢琴培训机构的小红书文案专家，擅长为不同年龄段的钢琴学习者创作吸引人的招生文案。

你的目标用户包括：
- 4-12岁的儿童及其家长（注重兴趣培养、基础启蒙）
- 13-18岁的青少年（关注考级、技能提升、升学加分）
- 大学生（追求个人兴趣发展、社交需求）
- 成人学习者（实现音乐梦想、减压放松、自我提升）

请始终采用'Thought-Action-Observation'模式进行推理。根据不同目标群体调整文案风格：
- 针对儿童家长：强调专业性、安全性、趣味性、教育价值
- 针对青少年：突出考级通过率、技能提升、同龄人社交
- 针对大学生：强调个性发展、音乐社团、校园生活丰富
- 针对成人：重点讲述圆梦、减压、自我实现、时间灵活

文案应该包含：吸引力标题、真实体验分享、课程特色介绍、成功案例、报名优惠信息。

完成任务后，请以JSON格式输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文（包含课程介绍、特色、优势等）",
  "hashtags": ["#钢琴培训", "#音乐教育", "#考级", "#成人学琴", "#钢琴启蒙"],
  "emojis": ["🎹", "🎵", "✨"]
}
```
"""

### 3.2 Tools (工具定义)

Agent 的“双手”由一系列可调用的工具组成。这些工具扩展了 LLM 的能力，使其能够获取实时信息、查询数据库或执行特定操作。在这里，我们定义了三个核心工具：

*   `search_web`: 用于搜索互联网上的实时信息，如最新趋势、用户评价等。
*   `query_product_database`: 用于查询产品数据库，获取产品的详细卖点和特点。**此工具为模拟**。
*   `generate_emoji`: 用于根据文案内容生成恰当的表情符号。**此工具为模拟**。

In [ ]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [5]:
# 添加钢琴培训相关的工具定义
PIANO_TRAINING_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search_piano_trends",
            "description": "搜索钢琴教育相关的最新趋势、热门话题、教学方法等信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "target_group": {
                        "type": "string",
                        "description": "目标群体，如'小学生'、'中学生'、'大学生'、'成人'、'钢琴初学者'等"
                    },
                    "topic": {
                        "type": "string", 
                        "description": "搜索主题，如'钢琴入门'、'考级技巧'、'钢琴练习方法'等"
                    }
                },
                "required": ["target_group", "topic"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_piano_course_info",
            "description": "获取钢琴培训课程的详细信息，包括课程内容、适合人群、教学特色等",
            "parameters": {
                "type": "object",
                "properties": {
                    "course_type": {
                        "type": "string",
                        "description": "课程类型，如'基础入门'、'考级辅导'、'成人速成'、'一对一教学'等"
                    },
                    "age_group": {
                        "type": "string",
                        "description": "年龄段，如'4-8岁'、'9-12岁'、'13-18岁'、'成人'等"
                    }
                },
                "required": ["course_type", "age_group"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_student_pain_points",
            "description": "分析不同年龄段学生在学钢琴过程中的痛点和需求",
            "parameters": {
                "type": "object",
                "properties": {
                    "student_type": {
                        "type": "string",
                        "description": "学生类型，如'钢琴零基础'、'有一定基础'、'准备考级'、'成人学习者'等"
                    }
                },
                "required": ["student_type"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_success_stories",
            "description": "获取钢琴培训的成功案例和学员故事",
            "parameters": {
                "type": "object",
                "properties": {
                    "achievement_type": {
                        "type": "string",
                        "description": "成就类型，如'考级通过'、'比赛获奖'、'快速入门'、'技能提升'等"
                    }
                },
                "required": ["achievement_type"]
            }
        }
    }
]

### 3.3 模拟工具实现

由于我们无法直接调用真实的外部 API (如Google Search或内部产品数据库)，我们将创建一些模拟 (Mock) 工具函数来演示 Agent 的工作流程。在实际应用中，您需要将这些模拟函数替换为真实的 API 调用。

In [4]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。"
    elif "美白精华" in product_name:
        return "美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

In [14]:
import time # 用于模拟网络延迟

# 钢琴培训相关的模拟工具实现
def mock_search_piano_trends(target_group: str, topic: str) -> str:
    """模拟搜索钢琴教育趋势"""
    print(f"[Tool Call] 搜索钢琴趋势 - 目标群体: {target_group}, 主题: {topic}")
    time.sleep(1)
    
    trends_data = {
        ("小学生", "钢琴入门"): "小学生钢琴学习热门话题：#趣味钢琴启蒙 #音乐游戏化教学 #亲子钢琴时光。家长关注点：培养兴趣、手指灵活性训练、音乐素养提升。",
        ("中学生", "考级技巧"): "中学生钢琴考级热门：#钢琴考级攻略 #技巧提升 #音乐理论。痛点：时间紧张、学业压力大、需要高效练习方法。",
        ("大学生", "钢琴练习方法"): "大学生钢琴学习趋势：#碎片化练习 #宿舍钢琴 #音乐社团。关注：时间管理、社交需求、个人兴趣发展。",
        ("成人", "钢琴入门"): "成人钢琴学习热点：#零基础学钢琴 #工作之余的音乐梦 #成人速成。需求：高效学习、实用曲目、减压放松。",
        ("钢琴初学者", "练习方法"): "钢琴初学者关注：#正确指法 #基础练习 #乐理入门。常见问题：手型不正确、练习枯燥、进步缓慢。"
    }
    
    key = (target_group, topic)
    return trends_data.get(key, f"关于{target_group}的{topic}，当前流行趋势包括个性化教学、在线学习结合线下指导、以及注重实践应用的教学方法。")

def mock_get_piano_course_info(course_type: str, age_group: str) -> str:
    """模拟获取钢琴课程信息"""
    print(f"[Tool Call] 获取课程信息 - 课程类型: {course_type}, 年龄段: {age_group}")
    time.sleep(0.8)
    
    course_info = {
        ("基础入门", "4-8岁"): "幼儿钢琴启蒙课程：采用奥尔夫音乐教学法，结合游戏化教学，培养音乐感知力。课程包括：手指操、简单儿歌、音符认识、节拍训练。小班教学，每节课45分钟。",
        ("基础入门", "9-12岁"): "少儿钢琴基础课程：系统学习钢琴基本功，包括五线谱、指法训练、简单乐曲演奏。采用中央音乐学院教材，为考级打基础。一对一或小组课可选。",
        ("考级辅导", "13-18岁"): "青少年钢琴考级课程：针对英皇、央音、上音等考级体系，专业老师一对一指导。包括技巧训练、曲目精讲、考试技巧。通过率95%以上。",
        ("成人速成", "成人"): "成人钢琴速成班：3个月学会弹奏经典曲目，课程设计实用高效。包括流行歌曲、经典名曲、即兴伴奏。线上线下结合，灵活安排时间。",
        ("一对一教学", "成人"): "成人一对一钢琴课：资深教师量身定制教学计划，根据个人兴趣和目标调整进度。包含乐理、技巧、曲目三大模块，学习效率提升300%。"
    }
    
    key = (course_type, age_group)
    return course_info.get(key, f"针对{age_group}的{course_type}课程，我们提供专业的钢琴教学服务，包括基础技能训练、音乐理论学习和实践演奏指导。")

def mock_analyze_student_pain_points(student_type: str) -> str:
    """模拟分析学生痛点"""
    print(f"[Tool Call] 分析学生痛点 - 学生类型: {student_type}")
    time.sleep(0.6)
    
    pain_points = {
        "钢琴零基础": "零基础学员痛点：不知道从何开始、担心年龄太大学不会、害怕枯燥练习、缺乏正确指导容易养成坏习惯。需要：循序渐进的教学方案、鼓励式教学、趣味性练习。",
        "有一定基础": "有基础学员痛点：技术瓶颈难突破、自学缺乏系统性、坏习惯需要纠正、缺乏专业指导。需要：个性化教学、技巧提升训练、系统化学习计划。",
        "准备考级": "考级学员痛点：考试压力大、技巧要求高、时间紧任务重、不熟悉考试流程。需要：针对性训练、模拟考试、心理辅导、高效练习方法。",
        "成人学习者": "成人学员痛点：时间有限、手指不够灵活、乐理基础薄弱、担心学习效果。需要：灵活上课时间、成人专用教材、快速入门方法、实用曲目练习。"
    }
    
    return pain_points.get(student_type, f"{student_type}的主要挑战包括学习时间安排、技能提升方法、以及持续学习动力的维持。")

def mock_get_success_stories(achievement_type: str) -> str:
    """模拟获取成功案例"""
    print(f"[Tool Call] 获取成功案例 - 成就类型: {achievement_type}")
    time.sleep(0.5)
    
    success_stories = {
        "考级通过": "学员小张(15岁)在我们这里学习8个月，从零基础到钢琴6级一次性通过！老师针对性训练，每天1小时高效练习，家长都说进步神速！",
        "比赛获奖": "成人学员李女士学琴2年，在上海市业余钢琴比赛中获得成人组二等奖！证明年龄不是问题，关键是科学的学习方法和坚持练习。",
        "快速入门": "零基础的王同学，仅用3个月就能弹奏《致爱丽丝》，现在已经成为班级里的钢琴小明星！我们的趣味教学法真的很有效。",
        "技能提升": "有基础的陈同学通过我们的进阶课程，钢琴技巧突飞猛进，现在能够演奏肖邦的高难度作品，还加入了学校音乐社团。"
    }
    
    return success_stories.get(achievement_type, f"我们有很多{achievement_type}的成功案例，学员们在专业指导下都取得了显著进步。")

# 将钢琴培训工具添加到可用工具字典
piano_tools = {
    "search_piano_trends": mock_search_piano_trends,
    "get_piano_course_info": mock_get_piano_course_info,
    "analyze_student_pain_points": mock_analyze_student_pain_points,
    "get_success_stories": mock_get_success_stories
}

## 4. 实战：构建小红书文案生成 Agent

现在，我们将把 System Prompt、工具定义和模拟工具函数整合起来，构建出能够自动执行的 DeepSeek Agent 工作流。核心是 `generate_rednote` 函数，它通过一个循环来模拟 Agent 的 `Thought-Action-Observation` 过程。

In [5]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]
    
    iteration_count = 0
    final_response = None
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION, # 告知模型可用的工具
                tool_choice="auto" # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message
            
            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史
                
                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")
                    
                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史
                
            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")
                
                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
                
                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    
    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

In [12]:
import json
import re

def generate_piano_training_rednote(target_audience: str, course_focus: str, tone_style: str = "专业亲和", max_iterations: int = 6) -> str:
    """
    生成钢琴培训的小红书文案
    
    Args:
        target_audience (str): 目标受众，如"小学生家长"、"中学生"、"大学生"、"成人"
        course_focus (str): 课程重点，如"零基础入门"、"考级辅导"、"技能提升"、"成人速成"
        tone_style (str): 文案风格
        max_iterations (int): 最大迭代次数
        
    Returns:
        str: 生成的钢琴培训文案（JSON格式）
    """
    
    print(f"\n🎹 启动钢琴培训文案生成助手")
    print(f"目标受众：{target_audience} | 课程重点：{course_focus} | 风格：{tone_style}\n")
    
    messages = [
        {"role": "system", "content": PIANO_TRAINING_SYSTEM_PROMPT},
        {"role": "user", "content": f"请为钢琴培训机构生成一篇小红书招生文案。目标受众：{target_audience}，课程重点：{course_focus}，文案风格：{tone_style}。请包含课程特色、学习效果、成功案例、报名信息等内容，并以完整的JSON格式输出。"}
    ]
    
    iteration_count = 0
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=PIANO_TRAINING_TOOLS,
                tool_choice="auto"
            )

            response_message = response.choices[0].message
            
            if response_message.tool_calls:
                print("Agent: 正在调用工具获取信息...")
                messages.append(response_message)
                
                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")
                    
                    if function_name in piano_tools:
                        tool_function = piano_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: {tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result)
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs)
                
            elif response_message.content:
                print(f"[模型生成结果] {response_message.content}")
                
                # JSON 提取和解析
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
                
                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 钢琴培训文案生成完成！")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: JSON解析失败: {e}")
                        messages.append(response_message)
                else:
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 钢琴培训文案生成完成！")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 继续优化文案内容...")
                        messages.append(response_message)
            else:
                print("Agent: 响应异常，尝试重新生成...")
                break
                
        except Exception as e:
            print(f"调用API时发生错误: {e}")
            break
    
    print("\n⚠️ 未能在指定迭代次数内生成完整的钢琴培训文案")
    return "文案生成失败，请重试。"

## 5. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [6]:
# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

-- Iteration 1 --
调用 DeepSeek API 时发生错误: Connection error.

⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。

--- 生成的文案 1 ---
未能成功生成文案。


In [7]:
# 测试案例 2: 美白精华
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '美白精华'}
[Tool Call] 模拟查询产品数据库：美白精华
Observation: 工具返回结果：美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '美白提亮、知性温柔'}
[Tool Call] 模拟生成表情符号，上下文：美白提亮、知性温柔
Observation: 工具返回结果：['🤩']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "✨一瓶美白精华=告别暗沉！烟酰胺+VC双管齐下，肌肤透亮如初✨",
  "body": "姐妹们！终于找到我的本命美白精华了！🤩\n\n最近熬夜加班，皮肤暗沉到不行，连粉底都盖不住那种黄气…直到用了这款美白精华，简直打开了新世界的大门！\n\n🌟核心成分是烟酰胺+VC衍生物，双管齐下，提亮肤色、淡化痘印效果超明显！\n🌟质地轻薄，吸收超快，完全不会黏腻，夏天用也毫无压力～\n🌟用了两周，皮肤真的透亮了好多，连同事都问我是不是换了粉底液！\n\n如果你也是暗沉肌、痘印困扰党，真的可以试试这款！温和不刺激，敏感肌也能放心用～\n\n#美白精华 #提亮肤色 #烟酰胺 #VC精华 #护肤好物",
  "hashtags": ["#美白精华", "#提亮肤色", "#烟酰胺", "#VC精华", "#护肤好物"],
  "emojis": ["✨", "🤩", "🌟", "💖", "🥰"]
}
```
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 2 ---
{
  "title": "✨一瓶美白精华=告别暗沉！烟酰胺+VC双管齐下，肌肤透亮如初✨",
  "body": "姐妹们！终于找到我的本命美白精华了！🤩\n\n最近熬夜加班，皮肤暗沉到不行，连粉

In [15]:
# 测试钢琴培训文案生成

# 测试案例1：针对小学生家长的钢琴启蒙课程
print("=== 测试案例1：小学生钢琴启蒙 ===")
piano_result_1 = generate_piano_training_rednote(
    target_audience="小学生家长",
    course_focus="零基础钢琴启蒙",
    tone_style="专业温暖"
)

print("\n--- 生成的钢琴培训文案 1 ---")
print(piano_result_1)

=== 测试案例1：小学生钢琴启蒙 ===

🎹 启动钢琴培训文案生成助手
目标受众：小学生家长 | 课程重点：零基础钢琴启蒙 | 风格：专业温暖

-- Iteration 1 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'search_piano_trends'，参数：{'target_group': '小学生', 'topic': '钢琴启蒙'}
[Tool Call] 搜索钢琴趋势 - 目标群体: 小学生, 主题: 钢琴启蒙
Observation: 关于小学生的钢琴启蒙，当前流行趋势包括个性化教学、在线学习结合线下指导、以及注重实践应用的教学方法。
-- Iteration 2 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'get_piano_course_info'，参数：{'course_type': '基础入门', 'age_group': '6-12岁'}
[Tool Call] 获取课程信息 - 课程类型: 基础入门, 年龄段: 6-12岁
Observation: 针对6-12岁的基础入门课程，我们提供专业的钢琴教学服务，包括基础技能训练、音乐理论学习和实践演奏指导。
-- Iteration 3 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'analyze_student_pain_points'，参数：{'student_type': '钢琴零基础'}
[Tool Call] 分析学生痛点 - 学生类型: 钢琴零基础
Observation: 零基础学员痛点：不知道从何开始、担心年龄太大学不会、害怕枯燥练习、缺乏正确指导容易养成坏习惯。需要：循序渐进的教学方案、鼓励式教学、趣味性练习。
-- Iteration 4 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'get_success_stories'，参数：{'achievement_type': '快速入门'}
[Tool Call] 获取成功案例 - 成就类型: 快速入门
Observation: 零基础的王同学，仅用3个月就能弹奏《致爱丽丝》，现在已经成为班级里的钢琴小明星！我们的趣味教学法真的很有效。
-- Iter

In [16]:
# 测试案例2：针对成人的钢琴速成课程
print("\n=== 测试案例2：成人钢琴速成 ===")
piano_result_2 = generate_piano_training_rednote(
    target_audience="成人",
    course_focus="成人钢琴速成",
    tone_style="励志温暖"
)

print("\n--- 生成的钢琴培训文案 2 ---")
print(piano_result_2)


=== 测试案例2：成人钢琴速成 ===

🎹 启动钢琴培训文案生成助手
目标受众：成人 | 课程重点：成人钢琴速成 | 风格：励志温暖

-- Iteration 1 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'get_piano_course_info'，参数：{'course_type': '成人速成', 'age_group': '成人'}
[Tool Call] 获取课程信息 - 课程类型: 成人速成, 年龄段: 成人
Observation: 成人钢琴速成班：3个月学会弹奏经典曲目，课程设计实用高效。包括流行歌曲、经典名曲、即兴伴奏。线上线下结合，灵活安排时间。
-- Iteration 2 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'analyze_student_pain_points'，参数：{'student_type': '成人学习者'}
[Tool Call] 分析学生痛点 - 学生类型: 成人学习者
Observation: 成人学员痛点：时间有限、手指不够灵活、乐理基础薄弱、担心学习效果。需要：灵活上课时间、成人专用教材、快速入门方法、实用曲目练习。
-- Iteration 3 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'get_success_stories'，参数：{'achievement_type': '快速入门'}
[Tool Call] 获取成功案例 - 成就类型: 快速入门
Observation: 零基础的王同学，仅用3个月就能弹奏《致爱丽丝》，现在已经成为班级里的钢琴小明星！我们的趣味教学法真的很有效。
-- Iteration 4 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'search_piano_trends'，参数：{'target_group': '成人', 'topic': '钢琴速成'}
[Tool Call] 搜索钢琴趋势 - 目标群体: 成人, 主题: 钢琴速成
Observation: 关于成人的钢琴速成，当前流行趋势包括个性化教学、在线学习结合线下指导、以及注重实践应用的教学方法。
-- Iteration 5 --
[模

In [17]:
# 测试案例3：针对中学生的考级辅导
print("\n=== 测试案例3：中学生考级辅导 ===")
piano_result_3 = generate_piano_training_rednote(
    target_audience="中学生",
    course_focus="钢琴考级辅导",
    tone_style="专业激励"
)

print("\n--- 生成的钢琴培训文案 3 ---")
print(piano_result_3)


=== 测试案例3：中学生考级辅导 ===

🎹 启动钢琴培训文案生成助手
目标受众：中学生 | 课程重点：钢琴考级辅导 | 风格：专业激励

-- Iteration 1 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'search_piano_trends'，参数：{'target_group': '中学生', 'topic': '钢琴考级辅导'}
[Tool Call] 搜索钢琴趋势 - 目标群体: 中学生, 主题: 钢琴考级辅导
Observation: 关于中学生的钢琴考级辅导，当前流行趋势包括个性化教学、在线学习结合线下指导、以及注重实践应用的教学方法。
-- Iteration 2 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'get_piano_course_info'，参数：{'course_type': '考级辅导', 'age_group': '13-18岁'}
[Tool Call] 获取课程信息 - 课程类型: 考级辅导, 年龄段: 13-18岁
Observation: 青少年钢琴考级课程：针对英皇、央音、上音等考级体系，专业老师一对一指导。包括技巧训练、曲目精讲、考试技巧。通过率95%以上。
-- Iteration 3 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'analyze_student_pain_points'，参数：{'student_type': '准备考级'}
[Tool Call] 分析学生痛点 - 学生类型: 准备考级
Observation: 考级学员痛点：考试压力大、技巧要求高、时间紧任务重、不熟悉考试流程。需要：针对性训练、模拟考试、心理辅导、高效练习方法。
-- Iteration 4 --
Agent: 正在调用工具获取信息...
Agent Action: 调用工具 'get_success_stories'，参数：{'achievement_type': '考级通过'}
[Tool Call] 获取成功案例 - 成就类型: 考级通过
Observation: 学员小张(15岁)在我们这里学习8个月，从零基础到钢琴6级一次性通过！老师针对性训练，每天1小时高效练习，家长都说进步神

### 格式化 小红书文案

**格式化函数 `format_rednote_for_markdown` 的功能：**

1. 解析 JSON 字符串。
2. 提取标题、正文、标签和表情符号。
3. 将它们组合成一个易读的 Markdown 格式的文本。


**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [8]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [9]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)


--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


In [11]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 💦深海蓝藻保湿面膜｜敏感肌救星！一夜回春的补水神器✨

姐妹们！我终于找到了我的本命面膜——深海蓝藻保湿面膜！💧

作为一个常年干燥+敏感肌的人，这款面膜简直就是我的救星！🌊核心成分是深海蓝藻提取物，富含多糖和氨基酸，敷完脸立马水润润的，而且一点都不粘腻！

🌟使用感：
- 精华液超多，敷完还能涂脖子和手臂！
- 质地清爽，敏感肌完全无负担！
- 第二天起床皮肤还是水嫩嫩的，上妆超服帖！

💖适合人群：
- 干燥肌、敏感肌的姐妹必入！
- 熬夜党、换季皮肤不稳定的宝子们！

真的是一夜回春的效果，我已经回购第三次了！姐妹们冲鸭！✨

#敏感肌救星 #深海蓝藻保湿面膜 #补水神器 #护肤必备 #面膜推荐


In [18]:
import json

# 格式化钢琴培训文案为易读格式
def format_piano_training_note(json_string: str) -> str:
    """
    将钢琴培训文案JSON格式转换为小红书发布格式
    """
    try:
        data = json.loads(json_string)
        title = data.get("title", "钢琴培训招生")
        body = data.get("body", "")
        hashtags = data.get("hashtags", [])
        
        # 构建最终发布格式
        formatted_note = f"## {title}\n\n{body}\n\n"
        
        if hashtags:
            hashtag_line = " ".join(hashtags)
            formatted_note += f"{hashtag_line}\n"
            
        return formatted_note.strip()
        
    except json.JSONDecodeError as e:
        return f"格式化失败: {e}\n原始内容:\n{json_string}"

# 示例：格式化第一个文案
if 'piano_result_1' in locals():
    formatted_piano_note_1 = format_piano_training_note(piano_result_1)
    print("\n=== 格式化后的钢琴培训文案 1 ===")
    print(formatted_piano_note_1)

# 示例：格式化第二个文案
if 'piano_result_2' in locals():
    formatted_piano_note_2 = format_piano_training_note(piano_result_2)
    print("\n=== 格式化后的钢琴培训文案 2 ===")
    print(formatted_piano_note_2)

# 示例：格式化第三个文案
if 'piano_result_3' in locals():
    formatted_piano_note_3 = format_piano_training_note(piano_result_3)
    print("\n=== 格式化后的钢琴培训文案 3 ===")
    print(formatted_piano_note_3)


=== 格式化后的钢琴培训文案 1 ===
## 🎹孩子学钢琴的最佳时机｜专业钢琴启蒙让宝贝爱上音乐

亲爱的家长朋友们～

您是否也在为孩子的兴趣培养而烦恼？

🌟【为什么选择我们】
• 专业钢琴教师团队，平均教龄8年+
• 针对6-12岁儿童研发的趣味教学法
• 小班制教学，确保每个孩子都得到充分关注
• 安全舒适的学习环境，家长可全程陪同

🎵【课程特色】
• 游戏化教学：通过音乐游戏激发学习兴趣
• 循序渐进：从认识键盘到弹奏简单曲目
• 音乐素养培养：同步学习乐理知识
• 表演机会：定期举办小型音乐会

✨【真实学员故事】
零基础的王同学，仅用3个月就能弹奏《致爱丽丝》，现在已经成为班级里的钢琴小明星！妈妈分享："没想到孩子这么喜欢，每天主动练琴！"

🎁【限时优惠】
• 新生体验课仅需99元（原价199元）
• 报名即赠精美乐理绘本+定制琴谱夹
• 前20名报名赠送价值300元音乐素养课

📍校区地址：XX区XX路XX号
📞咨询热线：400-XXX-XXXX

让音乐点亮孩子的童年，为宝贝开启艺术之门！

#钢琴启蒙 #儿童钢琴 #音乐教育 #小学生兴趣班 #钢琴培训 #音乐启蒙

=== 格式化后的钢琴培训文案 2 ===
## 30+学钢琴晚吗？3个月圆你音乐梦✨

💫『那个藏在心底的音乐梦想，现在开始也不晚』

还记得小时候路过琴行时羡慕的眼神吗？
工作多年后，终于鼓起勇气走进钢琴教室

🎹【成人钢琴速成班特色】
✅ 3个月学会弹奏经典曲目
✅ 线上线下灵活安排，工作党友好
✅ 成人专用教材，零基础也能快速上手
✅ 实用曲目教学，从流行歌曲到经典名曲

🌟【真实学员故事】
零基础的王同学，工作繁忙的职场人
仅用3个月就能完整弹奏《致爱丽丝》
现在已经成为朋友圈的"钢琴达人"

🎵【为什么选择我们】
• 个性化教学，根据你的进度调整
• 注重实践应用，学完就能弹给家人听
• 温暖的教学氛围，没有压力只有快乐
• 专业师资，耐心指导每个细节

💝【限时优惠】
现在报名即享：
🎁 免费体验课1节
🎁 教材费全免
🎁 前10名赠送精美琴谱

别再让梦想停留在想象中
从今天开始，让指尖流淌出美妙的旋律

📞 私信预约体验课，开启你的音乐之旅

#成人钢琴 #音乐梦想 #钢琴速成 #零基础学琴 #职场人充电 #自我提升 #音乐治愈

=== 格式化后的钢琴培

## 6. 评估与优化

文案生成并非一蹴而就，需要持续的评估和优化。本节讨论一些评估方法和优化策略。

#### 评估文案质量：
*   **客观量化评估 (数据)：**
    *   **点赞/收藏/评论/分享：** 基础互动
    *   **曝光/阅读/点击/涨粉：：** 流量与曝光
    *   **停留时长/截图率：** 用户行为。
    *   **商品页浏览/加购/ROI/成交转化：** 商业价值
    *   **爆文率/同类横向对比：** 竞争对比
*   **主观内部评估 (人工)：**
    *   **相关性：** 是否符合产品特点和主题。
    *   **吸引力：** 标题是否抓人，内容是否流畅。
    *   **合规性：** 是否有敏感词、违规宣传。
    *   **风格匹配：** 是否符合小红书调性和指定语气。
    *   **用户画像：** 目标人群年龄、地域、兴趣标签。



#### 优化迭代方法：
*   **Prompt 调整：** 根据评估结果，精修 System Prompt、User Prompt，增加或修改 Few-shot 示例。
*   **工具扩充：** 引入新的工具（如敏感词检测工具、竞品分析工具）。
*   **RAG (检索增强生成)：** 结合更精准的内部知识库，减少幻觉。


## 7. 总结与展望

通过本次实战，我们成功构建了一个基于 DeepSeek Agent 的小红书爆款文案生成助手。我们学习了如何拆解需求、设计 Prompt、定义工具，并实现 Agent 的核心工作流。

Agent 在内容营销领域的潜力巨大，未来可以进一步拓展到：

*   **超个性化内容：** 根据用户数据，生成一对一的定制文案。
*   **多模态内容创作：** 结合图片、视频生成，实现图文音视频一体化。
*   **智能营销决策：** Agent 不仅生成内容，还能分析效果并给出投放建议。
*   **跨平台适配：** 快速生成适应不同社交媒体平台风格的文案。

同时，我们也需关注挑战，如确保内容真实性、处理高度主观情感、与现有工作流的无缝集成等。Agent 技术仍在快速发展，期待未来能带来更多惊喜！